# Summary
This notebook provides a basic analysis of the regulation.gov comments to see where they come from and what kinds kinds of comments are present (mass campaign, attached documents, original comments, etc.)

In [2]:
import pandas
import numpy as np
import folium
from folium import plugins

/mnt/c/Users/nish/projects/titleix/env/lib/python3.5/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


The dataset of all comments loaded in from the Data Cleanup notebook.

In [51]:
dataset = pandas.read_json('./data/comments_with_attachments.json', orient='records', dtype='false')

Below is a print out of the dataset schema and the first comment.

In [53]:
print("Loaded %d comments without attachments." % len(dataset))
dataset.sample(1)

Loaded 13236 comments without attachments.


,doc.attachment_download,doc.attachment_download -href,doc.attachment_name,doc.category,doc.city,doc.comment_body,doc.country,doc.name,doc.state,doc.zip
6447,None,None,None,None,United States,"Dear Assistant General Counsel Hilary Malawer,...",Teacher,Janeen Godfrey,MI,49031


Below is a list of common strings to use in pattern matching. This list is handpicked and the matching is based on the one conducted [here](https://github.com/j2kao/fcc_nn_research/blob/master/proc_17_108_analysis_01_level_0_manual_tagging.ipynb) for FCC comments.
The common strings are used to create a dataset of only mass campaign comments and another dataset of comments without mass campaign comments.

In [7]:
common_strings = "Dear Assistant General Counsel Hilary Malawer,\n\nAll Department of Education civil rights regulations and guidance documents are important and necessary"
common_string2 = "Dear Assistant General Counsel Hilary Malawer,\n\nCurrent federal regulations and guidance help all studentsregardless of sex, race, color, sexual orientation,"

The `empty_comments` array is used to remove any comments that have no body from the datasets.

In [8]:
empty_comments = []
empty_comments.extend(comments_without_attachments[comments_without_attachments['doc.comment_body'] == ''].index.values)

#### From this we create 4 datasets
    1. All non-attachment comments (`all_comments`)
    2. Only mass campaign comments (`duplicate_comments`)
    3. Only non-mass compaign comments (`duplicates_removed`)
    4. Attachments (`attachments_only`)

In [59]:
all_comments = dataset.copy()

all_comments.drop(empty_comments, errors='ignore', inplace=True)

print("Total comments: ", len(all_comments))

Total comments:  12218


In [58]:
# Find rows containing the common strings
duplicate_comments = pandas.DataFrame()
duplicate_comments = duplicate_comments.append(dataset[dataset['doc.comment_body'].str.contains(common_strings, case=False, na=False)])
duplicate_comments = duplicate_comments.append(dataset[dataset['doc.comment_body'].str.contains(common_string2, case=False, na=False)])

duplicate_comments.drop(empty_comments, errors='ignore', inplace=True)

print("Duplicate comments: ", len(duplicate_comments))

Duplicate comments:  9357


In [61]:
duplicates_removed = dataset.copy()
duplicates_removed.drop(duplicate_comments.index.values, inplace=True)
duplicates_removed.drop(empty_comments, errors='ignore', inplace=True)

# Saving the dataset with duplicates removed for later use
duplicates_removed.to_json('./data/comments_duplicates_removed.json')

print("Non-duplicate comments: ", len(duplicates_removed))

Non-duplicate comments:  2861


In [62]:
attachments_only = pandas.read_json('./data/comments_only_attachments.json', orient='records', dtype='false')

print("Loaded %d attachments." % len(attachments_only))

Loaded 93 attachments.


In [57]:
# Reset the indexing of the datasets
all_comments = all_comments.reset_index(drop=True)
duplicate_comments = duplicate_comments.reset_index(drop=True)
duplicates_removed = duplicates_removed.reset_index(drop=True)
attachments_only = attachments_only.reset_index(drop=True)

### Below are samples comments from each type of dataset.

## `all_comments` sample

In [80]:
with pandas.option_context('display.max_colwidth', 5000):
    display(all_comments[['doc.comment_body']].sample(5))

,doc.comment_body
6703,"Dear Assistant General Counsel Hilary Malawer,\n\nAll Department of Education civil rights regulations and guidance documents are important and necessary. Far from being burdensome, current civil rights rules and regulations benefit schools and students by providing a clear framework that, when followed, allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex, race, color, national origin, disability status, English proficiency, sexual orientation, or gender identity.\nI urge the Department to keep in its current form 34 C.F.R. pts. 1 thru 1299 , which include regulations governing the Secretary and the offices for Civil Rights; Elementary and Secondary Education; Special Education and Rehabilitative Services; Career, Technical, and Adult Education; Post-Secondary Education; Educational Research and Improvement; and the National Council on Disability. \n\nI also urge the Department to preserve all current significant guidance documents, including guidance on sexual, racial, and disability-based harassment (including guidance on sexual violence); access to athletic opportunities; gender equity in career and technical education; single-sex schools; equal access to educational resources; nondiscriminatory school discipline; racial diversity programs; the rights of students with disabilities in charter schools; restraint and seclusion of students with disabilities; and the rights of English language learners. I urge you to keep current regulations and guidance in place, and to continue enforcing these critical civil rights laws so that all students have an equal opportunity to learn and thrive.\n\nSincerely,\nLucian Prevatt\n Hope Mills, NC 28348"
12696,"Dear Assistant General Counsel Hilary Malawer,\n\nAll Department of Education civil rights regulations and guidance documents are important and necessary. Far from being burdensome, current civil rights rules and regulations benefit schools and students by providing a clear framework that, when followed, allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex, race, color, national origin, disability status, English proficiency, sexual orientation, or gender identity.\nI urge the Department to keep in its current form 34 C.F.R. pts. 1 thru 1299 , which include regulations governing the Secretary and the offices for Civil Rights; Elementary and Secondary Education; Special Education and Rehabilitative Services; Career, Technical, and Adult Education; Post-Secondary Education; Educational Research and Improvement; and the National Council on Disability. \n\nI also urge the Department to preserve all current significant guidance documents, including guidance on sexual, racial, and disability-based harassment (including guidance on sexual violence); access to athletic opportunities; gender equity in career and technical education; single-sex schools; equal access to educational resources; nondiscriminatory school discipline; racial diversity programs; the rights of students with disabilities in charter schools; restraint and seclusion of students with disabilities; and the rights of English language learners. I urge you to keep current regulations and guidance in place, and to continue enforcing these critical civil rights laws so that all students have an equal opportunity to learn and thrive.\n\nSincerely,\nBarry Eshkol Adelman\n Vero Beach, FL 32967"
12095,"Dear Assistant General Counsel Hilary Malawer,\n\nAll Department of Education civil rights regulations and guidance documents are important and necessary. Far from being burdensome, current civil rights rules and regulations benefit schools and students by providing a clear framework that, when followed, allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex, race, color, national origin, disability status, English proficiency, sexual orientation, or gender identity

## `duplicate_comments` sample

In [82]:
with pandas.option_context('display.max_colwidth', 5000):
    display(duplicate_comments[['doc.comment_body']].sample(5))

,doc.comment_body
2089,"Dear Assistant General Counsel Hilary Malawer,\n\nAll Department of Education civil rights regulations and guidance documents are important and necessary. Far from being burdensome, current civil rights rules and regulations benefit schools and students by providing a clear framework that, when followed, allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex, race, color, national origin, disability status, English proficiency, sexual orientation, or gender identity.\nI urge the Department to keep in its current form 34 C.F.R. pts. 1 thru 1299 , which include regulations governing the Secretary and the offices for Civil Rights; Elementary and Secondary Education; Special Education and Rehabilitative Services; Career, Technical, and Adult Education; Post-Secondary Education; Educational Research and Improvement; and the National Council on Disability. \n\nI also urge the Department to preserve all current significant guidance documents, including guidance on sexual, racial, and disability-based harassment (including guidance on sexual violence); access to athletic opportunities; gender equity in career and technical education; single-sex schools; equal access to educational resources; nondiscriminatory school discipline; racial diversity programs; the rights of students with disabilities in charter schools; restraint and seclusion of students with disabilities; and the rights of English language learners. I urge you to keep current regulations and guidance in place, and to continue enforcing these critical civil rights laws so that all students have an equal opportunity to learn and thrive.\n\nSincerely,\nChristopher Lee\n Chicago, IL 60626"
10702,"Dear Assistant General Counsel Hilary Malawer,\n\nAll Department of Education civil rights regulations and guidance documents are important and necessary. Far from being burdensome, current civil rights rules and regulations benefit schools and students by providing a clear framework that, when followed, allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex, race, color, national origin, disability status, English proficiency, sexual orientation, or gender identity.\nI urge the Department to keep in its current form 34 C.F.R. pts. 1 thru 1299 , which include regulations governing the Secretary and the offices for Civil Rights; Elementary and Secondary Education; Special Education and Rehabilitative Services; Career, Technical, and Adult Education; Post-Secondary Education; Educational Research and Improvement; and the National Council on Disability. \n\nI also urge the Department to preserve all current significant guidance documents, including guidance on sexual, racial, and disability-based harassment (including guidance on sexual violence); access to athletic opportunities; gender equity in career and technical education; single-sex schools; equal access to educational resources; nondiscriminatory school discipline; racial diversity programs; the rights of students with disabilities in charter schools; restraint and seclusion of students with disabilities; and the rights of English language learners. I urge you to keep current regulations and guidance in place, and to continue enforcing these critical civil rights laws so that all students have an equal opportunity to learn and thrive.\n\nSincerely,\nKeith Curry\n Leander, TX 78641"
5311,"Dear Assistant General Counsel Hilary Malawer,\n\nAll Department of Education civil rights regulations and guidance documents are important and necessary. Far from being burdensome, current civil rights rules and regulations benefit schools and students by providing a clear framework that, when followed, allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex, race, color, national origin, disability status, English proficiency, sexual orientation, or gender identity.\nI urge the D

## `duplicates_removed` sample

In [84]:
with pandas.option_context('display.max_colwidth', 5000):
    display(duplicates_removed[['doc.comment_body']].sample(5))

doc.comment_body
8234   The Center for Community Self-Help welcomes the opportunity to comment on the U.S. Department of Education regulations and guidance pertaining to the Credit Enhancement for Charter School Facilities (CE) Program. Self-Help is a two-time CE program grantee, leveraging $10.2 million in grant funds into more than $300 million in loans over a 14-year period, helping schools create or maintain more than 42,000 student spaces.  The program has been a cornerstone for our charter school lending program and a vital tool for reaching borrowers in low-income communities that struggle with low collateral values and narrow operating margins. Fully 95% of our lending in the program has reached schools that serve low-income students and communities. Department of Education staff have been supportive partners in this effort, and we appreciate this opportunity for input and improvement.\n\nWe have also signed on to the comments submitted by the Charter School Lenders' Coalition (CSLC), and our suggestions largely echo those combined concerns. Self-Help and the 13 other members of the CSLC share a concern for increasing the efficiency of the CE program without reducing the end benefits to charter schools and the children they serve.\n\nOur comments include the following:\n\n1) Set a Reasonable Grant Period End\nSection F.20 of the Guidance document provides the following definition: "The grant period will run from the start date indicated on the grant award document until the Federal funds and earnings on those funds have been expended for the grant purposes or until financing facilitated by the grant has been retired, whichever is later."\n\nThis broad definition can cause the grant period to extend for decades, particularly for grantees like Self-Help that use the grant to back long-term debt and use the Reserve Account to guarantee additional debt as loans repay. Since the definition is in no way linked to performance goals, a grantee may have completed its initial grant period and far exceeded its financing and impact targets but remain subject to the same lengthy reporting requirements, including completion of the Annual Performance Report (APR), participation in quarterly monitoring calls, and time-consuming program audits. Self-Help, for example, is now four years beyond the original 10-year period of its 2003 CE grant and has more than tripled the $80 million lending goal. \n\nWe recommend that ongoing compliance and reporting instead be linked to the performance goals and time period formalized in the grant performance agreements. Compliance and reporting should conclude when all of the performance goals are achieved and the agreed-upon grant period has ended. At this point, the grantee has delivered on its contractual goals, greatly multiplied the impact of the grant dollars and maintained closely-monitored compliance over an extended period, demonstrating the ability to operate a long-term, high-impact charter school lending program.  We propose discussions with Department staff and charter school lending peers to explore ways in which these funds can be leveraged for maximum impact once the performance grant period hand s expired. \n\n2) Streamline Reporting Requirements\nThe Annual Performance Report excel spreadsheet currently requires the collection and reporting of 41 unique data points in separate fields for each school, with several fields requiring annual updates. We agree with our CSLC peers that the Department should conduct a comprehensive review to ensure the data collected is actually analyzed or used for a specific purpose or provides concrete value for compliance or program assessment.  We agree with the CSLC's list of reporting fields that can be modified or improved (below) and welcome further discussion on streamlining data collection: \n\nColumn C. Type of Transaction \n\nColumn H. Are then re any ongoing construction/facilities projects at this school \n\nColumn I. How were grant funds used

## `attachments_only` sample

In [85]:
with pandas.option_context('display.max_colwidth', 5000):
    display(attachments_only[['doc.comment_body']].sample(5))

doc.comment_body
47  The Honorable Betsy DeVos \nSecretary \nU.S. Department of Education \n400 Maryland Avenue SW \nWashington, DC 20202 \n \nRe: Docket ID ED-2017-OS-0074-0001 \n \nDear Secretary DeVos: \n \nTitle IX of the Education Amendments of 1972 is the federal law that prohibits sex \ndiscrimination in education. This vital law affects all areas of education, including: \nrecruitment, admissions and housing; pregnant and parenting students; science, \ntechnology, engineering, and math (STEM); sexual harassment and assault; and athletics. It \nrequires recipients of federal education funding to evaluate their current policies and \npractices, adopt and publish a policy against sex discrimination, and implement grievance \nprocedures providing for prompt and equitable resolution of student and employee \ndiscrimination complaints. \n \nI join with the American Association of University Women (AAUW) in urging the \nDepartment of Education to protect Title IX, preserve all of its current regulations and \nguidance, and fully enforce the law. \n \nUnfortunately, many students still do not have access to an equitable education free from \nsex discrimination. AAUW research found that 56 percent of girls and 40 percent of boys in \ngrades 7-12 face sexual harassment. Of that number, 87 percent said it had a negative effect \non them. In addition, girls have 1.2 million fewer chances to play sports in high school than \nboys. Less than two-thirds of African American and Hispanic girls play sports, while more \nthan three-quarters of Caucasian girls do. Pregnant and parenting students are often \nsteered toward separate, less rigorous schools. Just 12 percent of engineers are women, \nand the number of women in computing has fallen from 35 percent in 1990 to just 26 \npercent today. \n \nThe Office for Civil Rights (OCR) at the U.S. Department of Education has a long history of \nensuring technical assistance and fair enforcement of federal civil rights laws, including \nTitle IX. For decades, students and advocates have called on OCR to enforce Title IX, and its \nlong-standing regulations and guidance. \n \nAs Secretary of Education, you have the power to address this critical civil rights issue and \nhelp make schools safer and more equitable for all students. I join with AAUW in urging the \nDepartment of Education to keep in place current Title IX guidance and regulations and \nfully enforce the law. \n \nThank you, \n \nDonna-Lee ANDERSON  \n1633 Washington Blvd  \nStamford, CT 6902\n\n
The Honorable Betsy DeVos \nSecretary \nU.S. Department of Education \n400 Maryland Avenue SW \nWashington, DC 20202 \n \nRe: Docket ID ED-2017-OS-0074-0001 \n \nDear Secretary DeVos: \n \nTitle IX of the Education Amendments of 1972 is the federal law that prohibits sex \ndiscrimination in education. This vital law affects all areas of education, including: \nrecruitment, admissions and housing; pregnant and parenting students; science, \ntechnology, engineering, and math (STEM); sexual harassment and assault; and athletics. It \nrequires recipients of federal education funding to evaluate their current policies and \npractices, adopt and publish a policy against sex discrimination, and implement grievance \nprocedures providing for prompt and equitable resolution of student and employee \ndiscrimination complaints. \n \nI join with the American Association of University Women (AAUW) in urging the \nDepartment of Education to protect Title IX, preserve all of its current regulations and \nguidance, and fully enforce the law. \n \nUnfortunately, many students still do not have access to an equitable education free from \nsex discrimination. AAUW research found that 56 percent of girls and 40 percent of boys in \ngrades 7-12 face sexual harassment. Of that number, 87 percent said it had a negative effect \non them. In addition, girls have 1.2 million fewer chances to play sports in high school than \nboys. Less than two-thirds of African Am

## We take the 4 new datasets and plot them by state on a map. Each dataset is also broken down by zipcode in a table.

Importing the dataset containing the geographical data for states.

In [86]:
state_geo = './resources/us-states.json'

In [87]:
all_comments_map_data = all_comments[['doc.state', 'doc.zip']]
duplicate_comments_map_data = duplicate_comments[['doc.state', 'doc.zip']]
duplicates_removed_map_data = duplicates_removed[['doc.state', 'doc.zip']]

# Remove any null values
all_comments_map_data = all_comments_map_data.dropna();
duplicate_comments_map_data = duplicate_comments_map_data.dropna();
duplicates_removed_map_data = duplicates_removed_map_data.dropna();

Below is a function which will display the 5 most common states and zip codes from which the comments were sent for a dataset. It also displays a map of the the state data.

In [183]:
from IPython.core.display import HTML

def display_table_and_map(dataset_map_data):
    dataset_state_data = dataset_map_data.groupby(['doc.state']).size().reset_index(name='duplicate_count')
    dataset_state_data = dataset_state_data.sort_values('duplicate_count', ascending=False).reset_index(drop=True)
    dataset_zip_data = dataset_map_data.groupby(['doc.zip']).size().reset_index(name='duplicate_count')
    dataset_zip_data = dataset_zip_data.sort_values('duplicate_count', ascending=False).reset_index(drop=True)
    
    dataset_map = folium.Map(location=[48, -102], zoom_start=3)
    dataset_map.choropleth(
        geo_data=state_geo,
        name='map',
        data=dataset_state_data,
        columns=['doc.state', 'duplicate_count'],
        key_on='feature.id',
        fill_color='BuPu',
        fill_opacity=0.9,
        line_opacity=0.1
    )
    folium.LayerControl().add_to(dataset_map)
    
    return [display(HTML(
        '<table><tr style="background-color:white;">' + 
        '<td>' + dataset_state_data[:5]._repr_html_() + '</td>' +
        '<td>' + dataset_zip_data[:5]._repr_html_() + '</td>' +
        '</tr></table>'
    )), display(dataset_map)]

### The duplicate count by state and zip code for `all comments` as a table and map

In [188]:
display_table_and_map(all_comments);

,doc.state,duplicate_count
0,CA,1753
1,NY,895
2,WA,513
3,FL,489
4,PA,474
,doc.zip,duplicate_count
0,10025,18
1,11215,18
2,10011,15
3,10023,13


### The duplicate count by state and zip code for `duplicate comments` as a table and map

In [189]:
display_table_and_map(duplicate_comments);

,doc.state,duplicate_count
0,CA,1549
1,NY,757
2,WA,452
3,FL,435
4,PA,405
,doc.zip,duplicate_count
0,10025,17
1,11215,15
2,10011,13
3,10023,12


### The duplicate count by state and zip code for `duplicates removed` as a table and map

In [190]:
display_table_and_map(duplicates_removed);

,doc.state,duplicate_count
0,CA,204
1,NY,138
2,CO,127
3,DC,91
4,PA,69
,doc.zip,duplicate_count
0,20202,10
1,11215,3
2,98133,2
3,92115,2


In [192]:
display_table_and_map(attachments_only);

,doc.state,duplicate_count
0,DC,31
1,CA,4
2,AZ,3
3,MA,3
4,Parent/Relative,3
,doc.zip,duplicate_count
0,20202,2
1,13777,1


Resources:
https://github.com/j2kao/fcc_nn_research/blob/master/proc_17_108_analysis_01_level_0_manual_tagging.ipynb